# Crawling Emotional Words from Excel Data (Youtube Comments)

**유튜브 링크가 첨부된 엑셀 파일을 불러와서 해당 유튜브 링크 영상의 댓글로부터 추출한 감정 단어를 더해주는 코드**

한번 보시고, 이 방법으로 감정 단어를 추출하는 것이 괜찮은지 생각을...
- 품사가 '형용사'인 것만 뽑아도 댓글의 특성상 불용어가 너무 많다 (예외처리 비용 증가) -> 이 경우 불용어에 있는 규칙을 찾아서 아예 불용어 사전을 만들어버리는 방법이 있음.
- 크롤링을 통해 콘텐츠별로 감정 단어를 뽑아낼 수는 있으나, 콘텐츠별로 뚜렷한 경향성이 다르게 나올까? 라는 의문은 있음.

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from selenium import webdriver as wd
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

In [2]:
# 불용어 사전 불러오기
with open('stop_words.txt', 'r') as file:
    stop_word = file.readline()
    stop_word = str(stop_word)
    
stop_word = stop_word.replace("\ufeff", '').replace("'", '').replace(",", '').replace('\n', '').replace("’", '').replace("‘", '')
stop_words = stop_word.split()

In [3]:
# 곡 제목, 유튜브 링크가 있는 데이터 프레임(엑셀 파일)을 불러옴
data = pd.read_excel('/Users/lifeofpy/WebCrwaling/youtube_with_comments.xlsx')
data

,Song Title,Video Link,Emotional Words
0,Savage,https://www.youtube.com/watch?v=KJ2waPcG9OU,NaN
1,아낙네,https://www.youtube.com/watch?v=mi_UM-Prb1g,NaN
2,유엔 빌리지,https://www.youtube.com/watch?v=V8nu-jfka5E,NaN
3,낙하,https://www.youtube.com/watch?v=-bk9pR5IMm0,NaN
4,Toy,https://www.youtube.com/watch?v=i0wS2fVTYgg,NaN
5,So Hot (Blackpink),https://www.youtube.com/watch?v=kT7AIdX5cMI,NaN
6,Don't Call Me,https://www.youtube.com/watch?v=5-UHUjezfC8,NaN
7,시간을 달려서,https://www.youtube.com/watch?v=pN1LbNUFk8s,NaN
8,Dolphin,https://www.youtube.com/watch?v=3aEdigFwxkE,NaN
9,Make A Wish,https://www.youtube.com/watch?v=umM2zt0DH5g,NaN


In [4]:
data['Emotional Words'][0]  # 비어있는 nan 값의 Emotional Words 열을 감정 단어로 채우자!

nan

In [5]:
data_lst = data['Video Link'].tolist() # Series to List
data_lst

['https://www.youtube.com/watch?v=KJ2waPcG9OU',
 'https://www.youtube.com/watch?v=mi_UM-Prb1g',
 'https://www.youtube.com/watch?v=V8nu-jfka5E',
 'https://www.youtube.com/watch?v=-bk9pR5IMm0',
 'https://www.youtube.com/watch?v=i0wS2fVTYgg',
 'https://www.youtube.com/watch?v=kT7AIdX5cMI',
 'https://www.youtube.com/watch?v=5-UHUjezfC8',
 'https://www.youtube.com/watch?v=pN1LbNUFk8s',
 'https://www.youtube.com/watch?v=3aEdigFwxkE',
 'https://www.youtube.com/watch?v=umM2zt0DH5g']

In [6]:
for k in range(len(data)):
    driver = wd.Chrome(executable_path='/Users/lifeofpy/Desktop/chromedriver')
    url = data_lst[k] # url 에 영상 링크를 하나씩 담아준다
    driver.get(url)

    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")


    # 유튜브 프리미엄 유도 팝업창 피하기
    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass


    # 3초마다 자동 스크롤해 크롤링하기
    while True: 
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
        time.sleep(3.0) 
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight") 
        if new_page_height == last_page_height: 
            break 
        last_page_height = new_page_height 


    html_source = driver.page_source

    driver.close()

    soup = BeautifulSoup(html_source, 'lxml')
    
    # 댓글에 해당하는 html 태그 가져오기
    youtube_comments = soup.select('yt-formatted-string#content-text')
        
    okts = []
    
    for i in range(len(youtube_comments)):
        okts.append(okt.pos(youtube_comments[i].text))
    
    # '감정 단어'의 품사는 '형용사'이므로 형용사 태그만 adj 리스트에 담기
    adj = []

    for i in range(len(okts)):
        for j in range(len(okts[i])):
            if okts[i][j][1] == 'Adjective' and len(okts[i][j][0]) > 1:
                adj.append(okts[i][j][0])
                
    real_adj = []
    
    # 불용어 제외하고 real_adj 에 담기
    for w in adj:
        if w not in stop_words:
            real_adj.append(w)

    data.iloc[k, 2] = str(real_adj)

In [7]:
data.to_excel('Video_with_Emotional_Words.xlsx', index=False)